In [3]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

In [4]:
no_of_iteration = 100

In [71]:
INPUT_PATH= "/data/projects/shared_data/collab_data/10_transcripts"
INPUT_FILENAME = "lgg_gbm.csv"
OUTPUT_PATH = "/data/projects/shared_data/collab_data/10_transcripts/results"
OUTPUT_FILE_NAME = INPUT_FILENAME.split('.')[0]+"_empirical_p.csv"

In [72]:
df = pd.read_csv(INPUT_PATH+"/"+INPUT_FILENAME, sep= ";", decimal=',')
df= df.iloc[: , 1:]

In [73]:
df

,status,sample,ENST00000461347.5,ENST00000322723.8,ENST00000484328.1,ENST00000454824.5,ENST00000417652.5,ENST00000453992.5,ENST00000436894.1,ENST00000494618.1,ENST00000356936.5,ENST00000466274.1
0,gbm,TCGA-02-0047-01,1.940000,124.205770,-1.089298e-08,6.389815,6.847849e+01,9.999810e-02,-1.089298e-08,46.201532,49.521080,1.040022
1,gbm,TCGA-02-0055-01,2.059936,93.902043,-1.089298e-08,6.520031,1.199970e+00,1.299965e-01,-1.089298e-08,20.030520,45.739493,0.999971
2,gbm,TCGA-02-2483-01,1.700020,160.584743,1.299965e-01,2.949992,-1.089298e-08,-1.089298e-08,1.700038e-01,61.908976,134.988183,3.410012
3,gbm,TCGA-02-2485-01,2.840002,149.074566,-1.089298e-08,10.479968,2.092029e+01,2.199972e-01,8.100147e-01,47.510340,110.828644,4.580119
4,gbm,TCGA-06-0125-01,7.399896,173.728628,-1.089298e-08,22.420331,4.916878e+01,2.799918e-01,-1.089298e-08,45.209940,92.448909,4.740028
...,...,...,...,...,...,...,...,...,...,...,...,...
657,lgg,TCGA-WY-A85A-01,3.340046,140.097723,-1.089298e-08,14.950310,8.529912e+00,2.000045e-01,-1.089298e-08,20.729744,82.847289,2.569947
658,lgg,TCGA-WY-A85B-01,3.650021,182.125873,-1.089298e-08,19.879746,2.750084e+00,2.300056e-01,-1.089298e-08,43.621483,105.769977,4.029894
659,lgg,TCGA-WY-A85C-01,5.050108,134.530481,-1.089298e-08,6.209890,1.188013e+01,8.999935e-02,2.500071e-01,24.820183,80.297437,3.889889
660,lgg,TCGA-WY-A85D-01,3.389975,161.376988,-1.089298e-08,8.579728,8.499817e-01,1.200018e-01,-1.089298e-08,19.619637,112.726836,5.359992


In [74]:
print(df.groupby(['status']).size())
groups_name = list(df.groupby(['status']).groups.keys())
print(groups_name)

status
gbm    153
lgg    509
dtype: int64
['gbm', 'lgg']


In [75]:
def stratified_sample(df, strata, size=None, seed=None, keep_index= True):
    population = len(df)
    #print(population, size)
    size = __smpl_size(population, size)
    tmp = df.loc[:,strata]
    tmp['size'] = 1
    tmp_grpd = tmp.groupby(strata).count().reset_index()

    tmp_grpd['samp_size'] = np.round(size/population * tmp_grpd['size']).astype(int)

    # controlling variable to create the dataframe or append to it
    first = True 
    for i in range(len(tmp_grpd)):
        # query generator for each iteration
        qry=''
        for s in range(len(strata)):
            stratum = strata[s]
            value = tmp_grpd.iloc[i][stratum]
            n = tmp_grpd.iloc[i]['samp_size']

            if type(value) == str:
                value = "'" + str(value) + "'"
            
            if s != len(strata)-1:
                qry = qry + stratum + ' == ' + str(value) +' & '
            else:
                qry = qry + stratum + ' == ' + str(value)
        
        # final dataframe
        if first:
            stratified_df = df.query(qry).sample(n=n, random_state=seed).reset_index(drop=(not keep_index))
            first = False
        else:
            tmp_df = df.query(qry).sample(n=n, random_state=seed).reset_index(drop=(not keep_index))
            stratified_df = stratified_df.append(tmp_df, ignore_index=True)
    
    return stratified_df

In [76]:
def __smpl_size(population, size):
    '''
    A function to compute the sample size. If not informed, a sampling 
    size will be calculated using Cochran adjusted sampling formula:
        cochran_n = (Z**2 * p * q) /e**2
        where:
            - Z is the z-value. In this case we use 1.96 representing 95%
            - p is the estimated proportion of the population which has an
                attribute. In this case we use 0.5
            - q is 1-p
            - e is the margin of error
        This formula is adjusted as follows:
        adjusted_cochran = cochran_n / 1+((cochran_n -1)/N)
        where:
            - cochran_n = result of the previous formula
            - N is the population size
    '''
    if size is None:
        cochran_n = np.round(((1.96)**2 * 0.5 * 0.5)/ 0.02**2)
        n = np.round(cochran_n/(1+((cochran_n -1) /population)))
    elif size >= 0 and size < 1:
        n = np.round(population * size)
    elif size < 0:
        raise ValueError('Parameter "size" must be an integer or a proportion between 0 and 0.99.')
    elif size >= 1:
        n = size
    return n

In [77]:
temp_list = []
for i in range(no_of_iteration):
    temp_stratified = stratified_sample(df=df, strata=['status'], size=0.8)
    temp_list.append(temp_stratified)

/tmp/ipykernel_204331/2322919580.py:35: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  stratified_df = stratified_df.append(tmp_df, ignore_index=True)
/tmp/ipykernel_204331/2322919580.py:35: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  stratified_df = stratified_df.append(tmp_df, ignore_index=True)
/tmp/ipykernel_204331/2322919580.py:35: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  stratified_df = stratified_df.append(tmp_df, ignore_index=True)
/tmp/ipykernel_204331/2322919580.py:35: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  stratified_df = stratified_df.append(tmp_df, ignore_index=True)
/tmp/ipykernel_204331/2322919580.py:35: Futu

In [78]:
temp_list[0].dtypes

index                  int64
status                object
sample                object
ENST00000461347.5    float64
ENST00000322723.8    float64
ENST00000484328.1    float64
ENST00000454824.5    float64
ENST00000417652.5    float64
ENST00000453992.5    float64
ENST00000436894.1    float64
ENST00000494618.1    float64
ENST00000356936.5    float64
ENST00000466274.1    float64
dtype: object

In [79]:
df_p_values = pd.DataFrame()
df_p_values['Transcript_ID'] = df.columns[2:]
iteration =0
for df_temp in temp_list:
    iteration +=1
    if (iteration% 25) ==0:
        print(iteration)
    #print(df_temp.groupby(['lung_tissue']).size())                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        
    cat1 = df_temp[df_temp['status']==groups_name[0]]
    cat2 = df_temp[df_temp['status']==groups_name[1]]
    for index, row in df_p_values.iterrows():
        tuple_values = ttest_ind(cat1[row['Transcript_ID']], cat2[row['Transcript_ID']])
        df_p_values.loc[index, ['ITER_{}'.format(iteration)]]= tuple_values[1]
        index+=1

25
50
75
100


In [80]:
df_p_values.set_index('Transcript_ID', inplace=True)
df_p_values

,ITER_1,ITER_2,ITER_3,ITER_4,ITER_5,ITER_6,ITER_7,ITER_8,ITER_9,ITER_10,...,ITER_91,ITER_92,ITER_93,ITER_94,ITER_95,ITER_96,ITER_97,ITER_98,ITER_99,ITER_100
Transcript_ID,,,,,,,,,,,,,,,,,,,,,
ENST00000461347.5,5.616870e-01,8.680863e-01,6.937453e-01,5.924944e-01,6.458492e-01,2.441160e-01,7.340324e-01,5.630737e-01,1.673342e-01,7.652180e-01,...,5.999042e-01,4.137992e-01,4.787739e-01,4.851202e-01,7.676317e-01,5.591574e-01,5.826838e-01,4.599403e-01,1.572574e-01,5.868887e-01
ENST00000322723.8,1.061737e-24,1.354724e-20,1.251753e-22,1.512047e-26,1.127951e-23,6.447188e-26,1.491565e-23,1.533697e-27,2.495226e-27,1.141843e-20,...,1.833605e-23,7.165981e-27,2.246412e-26,3.253953e-24,1.477992e-23,4.326672e-25,5.231333e-23,6.177096e-26,3.794921e-23,8.249898e-23
ENST00000484328.1,2.392597e-02,2.344372e-02,1.279214e-01,7.097209e-02,5.931655e-02,3.268706e-01,7.326361e-02,8.784353e-03,3.591462e-01,3.981999e-02,...,9.996963e-03,3.940316e-01,1.703720e-01,2.369782e-01,3.762694e-02,2.413304e-01,3.990986e-01,4.075931e-01,9.234382e-02,6.767852e-02
ENST00000454824.5,2.590650e-07,1.318724e-06,2.899902e-05,1.211347e-08,2.039716e-08,8.599160e-07,9.671865e-06,1.439511e-08,3.556258e-07,5.096173e-06,...,5.813703e-05,5.713810e-08,1.112736e-06,6.833020e-06,4.112972e-07,4.712073e-07,3.154828e-06,1.117207e-07,2.166169e-06,1.347781e-05
ENST00000417652.5,6.275096e-03,2.761588e-02,5.271899e-02,1.680212e-02,1.816238e-02,5.615686e-03,2.716380e-02,6.978005e-03,4.709332e-03,2.783022e-02,...,8.049971e-02,3.634081e-02,3.665022e-03,3.190477e-02,6.517598e-03,8.859745e-03,7.409672e-02,1.757236e-03,5.212081e-02,3.787696e-02
ENST00000453992.5,2.030765e-05,5.235815e-05,7.107863e-06,3.628180e-07,2.972383e-06,2.603416e-06,1.314122e-05,2.463010e-06,2.886536e-06,8.945793e-06,...,6.793902e-05,9.619424e-07,4.238288e-06,1.061007e-06,1.220641e-05,2.521327e-06,8.743815e-05,1.140817e-06,1.168269e-06,2.941764e-05
ENST00000436894.1,1.473342e-01,5.237632e-01,1.705957e-01,3.537095e-01,3.756218e-01,2.822741e-01,6.183190e-01,5.634802e-01,8.240217e-01,1.023445e-01,...,3.472870e-01,2.895399e-01,2.766786e-01,2.456090e-01,4.164067e-01,4.845294e-01,9.019953e-02,2.908221e-01,1.194253e-01,7.319565e-01
ENST00000494618.1,7.366872e-01,2.718498e-01,5.181304e-01,9.975807e-01,9.505984e-01,9.163348e-01,5.043311e-01,5.928942e-01,4.666788e-01,5.320635e-01,...,2.716773e-01,8.543642e-01,9.661081e-01,9.019145e-01,7.885748e-01,9.596686e-01,5.095825e-01,5.772141e-01,6.156931e-01,4.671112e-01
ENST00000356936.5,1.845342e-01,1.022715e-02,9.032025e-02,1.515594e-01,1.060627e-01,1.011477e-01,2.032233e-01,2.147184e-01,5.888284e-01,2.643916e-02,...,2.830473e-01,5.711996e-01,2.342092e-01,2.621432e-01,7.891851e-03,5.274625e-01,5.351161e-02,6.891780e-02,1.645187e-01,1.089156e-01


In [81]:
def count(df):
    #n=df.shape[1]
    #print(df)
    threshold_values=[0.05]
    for threshold in threshold_values:
        r = np.sum(df >= threshold)
        df['>={}'.format(threshold)] = (r+1) /(no_of_iteration+1)
    #df['r'] = np.sum(df == 'r')
    return df

In [82]:
temp_df_empirical = df_p_values.apply(count, axis = 1)

In [83]:
temp_df_empirical.iloc[: , -1:]

,>=0.05
Transcript_ID,
ENST00000461347.5,1.000000
ENST00000322723.8,0.009901
ENST00000484328.1,0.653465
ENST00000454824.5,0.009901
ENST00000417652.5,0.198020
ENST00000453992.5,0.009901
ENST00000436894.1,1.000000
ENST00000494618.1,1.000000
ENST00000356936.5,0.811881


In [84]:
df_mean = df.groupby("status").mean(numeric_only=True)
df_mean

,ENST00000461347.5,ENST00000322723.8,ENST00000484328.1,ENST00000454824.5,ENST00000417652.5,ENST00000453992.5,ENST00000436894.1,ENST00000494618.1,ENST00000356936.5,ENST00000466274.1
status,,,,,,,,,,
gbm,2.900284,117.152197,0.090392,10.418378,21.412329,0.171307,0.046667,32.872318,70.108631,2.412982
lgg,3.018012,166.250002,0.068271,15.472334,27.718611,0.274204,0.037623,32.210526,65.604207,1.845578


In [85]:
index_list = list(df_mean.index.values)

In [86]:
df_mean.loc['Fold_change_'+index_list[0]+'/'+index_list[1]] = df_mean.iloc[0]/df_mean.iloc[1]
df_mean.loc['Fold_change_'+index_list[1]+'/'+index_list[0]] = df_mean.iloc[1]/df_mean.iloc[0]
df_mean

,ENST00000461347.5,ENST00000322723.8,ENST00000484328.1,ENST00000454824.5,ENST00000417652.5,ENST00000453992.5,ENST00000436894.1,ENST00000494618.1,ENST00000356936.5,ENST00000466274.1
status,,,,,,,,,,
gbm,2.900284,117.152197,0.090392,10.418378,21.412329,0.171307,0.046667,32.872318,70.108631,2.412982
lgg,3.018012,166.250002,0.068271,15.472334,27.718611,0.274204,0.037623,32.210526,65.604207,1.845578
Fold_change_gbm/lgg,0.960992,0.704675,1.324019,0.673355,0.772489,0.624743,1.240386,1.020546,1.068661,1.307440
Fold_change_lgg/gbm,1.040592,1.419094,0.755276,1.485100,1.294516,1.600659,0.806201,0.979868,0.935751,0.764854


In [87]:
temp_df_empirical['Fold_change_gbm/lgg'] = df_mean.loc['Fold_change_gbm/lgg'].values[:]
temp_df_empirical['Fold_change_lgg/gbm'] = df_mean.loc['Fold_change_lgg/gbm'].values[:]

In [88]:
temp_df_empirical

,ITER_1,ITER_2,ITER_3,ITER_4,ITER_5,ITER_6,ITER_7,ITER_8,ITER_9,ITER_10,...,ITER_94,ITER_95,ITER_96,ITER_97,ITER_98,ITER_99,ITER_100,>=0.05,Fold_change_gbm/lgg,Fold_change_lgg/gbm
Transcript_ID,,,,,,,,,,,,,,,,,,,,,
ENST00000461347.5,5.616870e-01,8.680863e-01,6.937453e-01,5.924944e-01,6.458492e-01,2.441160e-01,7.340324e-01,5.630737e-01,1.673342e-01,7.652180e-01,...,4.851202e-01,7.676317e-01,5.591574e-01,5.826838e-01,4.599403e-01,1.572574e-01,5.868887e-01,1.000000,0.960992,1.040592
ENST00000322723.8,1.061737e-24,1.354724e-20,1.251753e-22,1.512047e-26,1.127951e-23,6.447188e-26,1.491565e-23,1.533697e-27,2.495226e-27,1.141843e-20,...,3.253953e-24,1.477992e-23,4.326672e-25,5.231333e-23,6.177096e-26,3.794921e-23,8.249898e-23,0.009901,0.704675,1.419094
ENST00000484328.1,2.392597e-02,2.344372e-02,1.279214e-01,7.097209e-02,5.931655e-02,3.268706e-01,7.326361e-02,8.784353e-03,3.591462e-01,3.981999e-02,...,2.369782e-01,3.762694e-02,2.413304e-01,3.990986e-01,4.075931e-01,9.234382e-02,6.767852e-02,0.653465,1.324019,0.755276
ENST00000454824.5,2.590650e-07,1.318724e-06,2.899902e-05,1.211347e-08,2.039716e-08,8.599160e-07,9.671865e-06,1.439511e-08,3.556258e-07,5.096173e-06,...,6.833020e-06,4.112972e-07,4.712073e-07,3.154828e-06,1.117207e-07,2.166169e-06,1.347781e-05,0.009901,0.673355,1.485100
ENST00000417652.5,6.275096e-03,2.761588e-02,5.271899e-02,1.680212e-02,1.816238e-02,5.615686e-03,2.716380e-02,6.978005e-03,4.709332e-03,2.783022e-02,...,3.190477e-02,6.517598e-03,8.859745e-03,7.409672e-02,1.757236e-03,5.212081e-02,3.787696e-02,0.198020,0.772489,1.294516
ENST00000453992.5,2.030765e-05,5.235815e-05,7.107863e-06,3.628180e-07,2.972383e-06,2.603416e-06,1.314122e-05,2.463010e-06,2.886536e-06,8.945793e-06,...,1.061007e-06,1.220641e-05,2.521327e-06,8.743815e-05,1.140817e-06,1.168269e-06,2.941764e-05,0.009901,0.624743,1.600659
ENST00000436894.1,1.473342e-01,5.237632e-01,1.705957e-01,3.537095e-01,3.756218e-01,2.822741e-01,6.183190e-01,5.634802e-01,8.240217e-01,1.023445e-01,...,2.456090e-01,4.164067e-01,4.845294e-01,9.019953e-02,2.908221e-01,1.194253e-01,7.319565e-01,1.000000,1.240386,0.806201
ENST00000494618.1,7.366872e-01,2.718498e-01,5.181304e-01,9.975807e-01,9.505984e-01,9.163348e-01,5.043311e-01,5.928942e-01,4.666788e-01,5.320635e-01,...,9.019145e-01,7.885748e-01,9.596686e-01,5.095825e-01,5.772141e-01,6.156931e-01,4.671112e-01,1.000000,1.020546,0.979868
ENST00000356936.5,1.845342e-01,1.022715e-02,9.032025e-02,1.515594e-01,1.060627e-01,1.011477e-01,2.032233e-01,2.147184e-01,5.888284e-01,2.643916e-02,...,2.621432e-01,7.891851e-03,5.274625e-01,5.351161e-02,6.891780e-02,1.645187e-01,1.089156e-01,0.811881,1.068661,0.935751


In [89]:
temp_df_empirical.iloc[: , -3:]

,>=0.05,Fold_change_gbm/lgg,Fold_change_lgg/gbm
Transcript_ID,,,
ENST00000461347.5,1.000000,0.960992,1.040592
ENST00000322723.8,0.009901,0.704675,1.419094
ENST00000484328.1,0.653465,1.324019,0.755276
ENST00000454824.5,0.009901,0.673355,1.485100
ENST00000417652.5,0.198020,0.772489,1.294516
ENST00000453992.5,0.009901,0.624743,1.600659
ENST00000436894.1,1.000000,1.240386,0.806201
ENST00000494618.1,1.000000,1.020546,0.979868
ENST00000356936.5,0.811881,1.068661,0.935751


In [70]:
temp_df_empirical.iloc[: , -3:].to_csv(OUTPUT_PATH+"/"+OUTPUT_FILE_NAME, sep=",")